In [17]:
import sympy as sp

from symqups.objects.scalars import q, p, alpha, alphaD, W
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho
from symqups.utils.algebra import define

In [ ]:
a = annihilateOp()
ad = createOp()
x = sp.Symbol("x")

O(\hat{a}^{\dagger}_{}*\hat{a}_{}*\hat{a}^{\dagger}_{}*\hat{a}_{}*\hat{a}^{\dagger}_{}*\hat{a}_{}*\hat{a}^{\dagger}_{}*\hat{a}_{}*\hat{a}^{\dagger}_{}*\hat{a}_{}*\hat{a}^{\dagger}_{}*\hat{a}_{}, \hat{a}_{}, \hat{a}^{\dagger}_{})